<a href="https://colab.research.google.com/github/N111N3/kaggle_courses/blob/master/pandas/exercise1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'pitchfork-data:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F655%2F1252%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240327%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240327T042344Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D0979f8c14a2a8093ad9fae329351d86daa1537244ce77a4479205ee2adc752045f4225326a8dc9d0bd298273efc9d6d420089808a6735654a7e0559dc638ab6f1ab8acd4174ff6986762a2b2438468970e47eb5cd8c2b22b3305236ea985ba4448eed37436663ad51c168a8b21f422ef4d30bb3d8e4f3f21df325bc3b45df646376c03e3921d8f051e06b9848df9917c519fef26b5d700ab619fa945a2bb537eac461aa5844cee4e792b229b5f68a4e6e69fcf5cae780fd356b29da9f6e36d49e63aebf65e15efe7db632e74881b022cadb50d58341045df07b30e5e17ab4d706016596424d921a664e195aeb3bce176fc77d3e2b66a4157599e4ca60faf01f4,chess:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2321%2F3919%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240327%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240327T042344Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D5c7679581a2bb6a05734a598aa06900fca0adddab8e9634f566cdcd2173ef90cea2806167e9c157edb2d166eca6ec08cf7c906d050bf6aecbccc7905592191490c3d486b295ee5ed1bca3dfc0b3d38098c358f5eac2f6138709f8f6036be8fd5c913485ab413e0fa1fdecd8b6bd8ef1ad89ae1764b1e472ffc5e04f34eb431fcf4981b6f18cb114ed1605b12be30eab35425a6bbb93c5a989060728b8cd3e1c9725d5aa4d948750b39da10790bb6a03816c7869a4474c2c2418b673bf34977f27ad133d4f8b4c428219d00c7535bc344ff8f0c1961a2f1d7a4caa039635d603bccedf957a0f68777a1f4ffdb2911f6463f1ec63c3672376eacddac5dfa16f641,kepler-exoplanet-search-results:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2894%2F4877%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240327%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240327T042344Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D7fd4391689a9715c5af7c008d10ce9e3eef94971bf6981736387c224ff2bc906bfa282a48799bf70cdcb176b6465763093ab5381b9ab9479007664e944a482ef63b8532ebafece00252463dd7f6a58f006b6e4aee845a578cddbe2a7b4ed898cc6514ae2a8c5db644ee87b04597e3d6131618ff7a705a578590c03871bdc3ce950babc9db7730166d4acc48e1b9966ece49684062d72ac3757ddb01581ff10c2fb70dd161db1538c33dbd339f754b32b119b7385d78e6c7812602c907ad8def2a1db2259e2730bd178b349e984118da281d9a78316a37902a7b485b201bd73c944e9da2fdceb5ac0a429be9babb27e7d52421a33a949f04e2e996bdbf1d6c6aa,things-on-reddit:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F3491%2F5624%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240327%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240327T042344Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D8710bdfa59c87f7ac8fe88613978b2e45a559b3383fdf1d910a92a9577142d842b95a79711c71baf825bc337331d823a322cf90573cd25a3b3d022a9d2120b336efc5a34dead74295c2f64f65dcdf14ae4b4265cb510e5f23325e24bc5e750f56a2914614aca3501a7c2c999392a954783bf645ba0dfd807bb773a0d1f62eabe36d1bb470aa5a95fa1595d937a4b0d3be4f6cd498e1ed721b87254e2b70a05e9fd5805389155bc09682c3cbc715061d09de3de8e6c5200902463e4968a327c9e29f39e972a0fccf28683c5a156c2a7fa4176d8be0e6918f506ab2866369e72aa25b859354a3c2054a8b4186cb09540147653601b1192668e59dcf1e1fd6224af,wine-reviews:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1442%2F8172%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240327%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240327T042344Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D0c7befadeb91cd0f8852eee0215515f8432e352cb02b338f6fe32c921bcfe4753ba68fe09e3069f9983488fd51f38fbeb152c5660c1d2c6ae9b30223425a873318a8bb6ca08328a8b5cf265badb91716e6af89f3bf89cb7838a27b44dba88a10642481c3b14a60c4cbfa9c1831afb0ee952b1ea845fae5887d5f0b791f350c62c94dc444c9013cb4ea1dd709651561b8a79309b2a15af7c25578209948982c30f0ef0219c59e80b9f60597e9b854a7cff91c333e4b85db35972ee57280f0b272628d5626f4ee78ec1d31e94dc2c89c6b57fc7070f550ddd91d17934f9c50ac2444ec1d5d8990618bd8b554d416cbb8621757dfa93d707ea4b1e51b72b5ace471,ramen-ratings:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F9366%2F13206%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240327%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240327T042344Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D572ede7cb94cc98103829c48e4c88b34f86298641cd46b104e00924221432ca28aa03553cd87d90d06fc9fd304383be6e97cb63abb8e04a6c741e777133634094c9a83b73f4bad87eb7f4f633d9d22c7ecc580bba6c403b9ae6048352dcb238c7cfab259f5fa1cd4343cfeb3f29b6e3b0f50eff7b26d68dc0106ba24c06aa6a11fb1eb980f63326b8e3b79101fc15ec809de2a963769ee2b007c17c83bd25c9278b4ed787f8b595443ac09c820fbb0e4d2a6ed271b1e5bc1de6fc79881b47ddbcd750c2076900c8712f58bed9a18fb17e3e5b47ff146c33efcc6bb39d5850c28c503f29c70b73704f55499f81eaed82e88c29a08cedf33f0a7f838830df422f0,powerlifting-database:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F179555%2F403916%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240327%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240327T042344Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D7005adf4a97f7856a2e46d42d342640367c56137a1bf67ba774cca9374e22a5c9b71a399b75033358b2165160f9b3b219d8df85cc058cc1a61dfe73a6aa841933e81c95a42228f61e7189c4f8f2c865f559c35c708d963ae23fdfc315379d1ab1131ad2f2d8db6470225b3cb18adaf994289d5e3892ea9ee799aafac506a827e63918b66b720ef4700c6cd04ab95d720c110c71b9930d722ba4b984f2f9ee8b0bb95da72c6fafa21945e1b455db562790793001c7f00dbe1f39ec9f19201e9c559727e014f7a8412cecf74ddbc241b33cdc94f3bcbe2d10eb79658ae415f85602e45527689b585cc77683e3db1e83afc313669f6259951f56ae48afff2e3f67d,youtube-new:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4549%2F466349%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240327%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240327T042344Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D7435a4fdb8933692e1eebe7ad2f402d1badc646a78992909d2753eef47568fc246cb973565e688c3ff73101e866732612a2d54eb5907f9b558a14bcb94d7caf81f7ea08db75c983a4c66e272d75b43ad12d0fd560986fe11ac1010fbce19efa8395730487d9627727985ba1e4e36ef2afe933804e11597a6e6cb055bdd3cf2da2ed4ff0166426595aaafff8077b0232f74bb65a64c96dee34a29f8070455e4944b1d763585696751558567f3f6b6df2174944a3900d3e62d099fefae4e9859fb283e31382adfa10fde115c709100f00c10c6232c0e4b024fa522082c9614debd5cccf08d30a1353615ece921e35b2bd14594b4ddd2f65117d64d777b79670338,188-million-us-wildfires:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2478%2F1151655%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240327%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240327T042344Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D569b09fc611741c343e65378332b0d15db15d870108146998b7dec24fc19e0a39d54fcffa4b686b1a50bd2a4a8eee365f3341f9a2daf2b2e3b0ce7239acd5ec6a2c435424444b24236a41902c08a87b8b6708e535902388e955e3746246680f46ccfd483877745d69d22e9b35b271a0a7bbd044e6d2b1b642ad486626bde2cb1a1d86a5726d5d10224e4969236f4074296e0d4dd1a71d1916103d5ab2ff7a8113a2ece56309dd2bedb55a854138e0d6b16f75b366db977ea059fe33f1616e6af85bae60c43c5b32ecbf8b46da4a6d409a4b09a58bf005d4794c8ee4d5c06be18b1aacc1102c3010435f9130fd638a2a94f53c31ac98c8ba274b55eee631895bc,publicassistance:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F10128%2F5438389%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240327%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240327T042344Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D0eb347a5dffd03c51fd52940d76be8a2cda073a677d1ce98743df620cbeb68eb551294e63cf5aa29ed7e93500ff0157d8c91149e166a09e63fb4bfcf20621dcc5d2d69e77cab193783201ec5bf2d406861504d8c6e90316c87d7e24a4411975adb7733d0de052e34853ccc9f1729367f82cb7298a9bea339e17bd48d407bcfdbaec630c0f17852b9ff1e6afb11829c5b3ecd53d263b677315ba577bb34aa5220b2d26b8bf753584618f5bccd9b2ffe2a7c1bbdadde0f688d1d9a44529ac693f58b10935b9ba83996e815e83b6b1030781c085d2cbcbe39cb0954c19da2da6c0fe0fa0f6f624faaff6b9fcf168fd1452c68d77527dd1af5a32554371ab1aa3d27'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


**This notebook is an exercise in the [Pandas](https://www.kaggle.com/learn/pandas) course.  You can reference the tutorial at [this link](https://www.kaggle.com/residentmario/creating-reading-and-writing).**

---


# Introduction

The first step in most data analytics projects is reading the data file. In this exercise, you'll create Series and DataFrame objects, both by hand and by reading data files.

Run the code cell below to load libraries you will need (including code to check your answers).

In [ ]:
import pandas as pd
pd.set_option('display.max_rows', 5)
from learntools.core import binder; binder.bind(globals())
from learntools.pandas.creating_reading_and_writing import *
print("Setup complete.")

# Exercises

## 1.

In the cell below, create a DataFrame `fruits` that looks like this:

![](https://storage.googleapis.com/kaggle-media/learn/images/Ax3pp2A.png)

In [ ]:
# Your code goes here. Create a dataframe matching the above diagram and assign it to the variable fruits.
fruits = pd.DataFrame({'Apples':[30],'Bananas':[21]})

# Check your answer
q1.check()
fruits

In [ ]:
#q1.hint()
#q1.solution()

## 2.

Create a dataframe `fruit_sales` that matches the diagram below:

![](https://storage.googleapis.com/kaggle-media/learn/images/CHPn7ZF.png)

In [ ]:
# Your code goes here. Create a dataframe matching the above diagram and assign it to the variable fruit_sales.
fruit_sales = pd.DataFrame({'Apples':[35,41], 'Bananas':[21,34]}, index=['2017 Sales', '2018 Sales'])

# Check your answer
q2.check()
fruit_sales

In [ ]:
#q2.hint()
#q2.solution()

## 3.

Create a variable `ingredients` with a Series that looks like:

```
Flour     4 cups
Milk       1 cup
Eggs     2 large
Spam       1 can
Name: Dinner, dtype: object
```

In [ ]:
ingredients = pd.Series(['4 cups', '1 cup', '2 large', '1 can'], index=['Flour', 'Milk', 'Eggs', 'Spam'], name='Dinner')

# Check your answer
q3.check()
ingredients

In [ ]:
#q3.hint()
#q3.solution()

## 4.

Read the following csv dataset of wine reviews into a DataFrame called `reviews`:

![](https://storage.googleapis.com/kaggle-media/learn/images/74RCZtU.png)

The filepath to the csv file is `../input/wine-reviews/winemag-data_first150k.csv`. The first few lines look like:

```
,country,description,designation,points,price,province,region_1,region_2,variety,winery
0,US,"This tremendous 100% varietal wine[...]",Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
1,Spain,"Ripe aromas of fig, blackberry and[...]",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,,Tinta de Toro,Bodega Carmen Rodríguez
```

In [ ]:
reviews = pd.read_csv('../input/wine-reviews/winemag-data_first150k.csv', index_col=0)

# Check your answer
q4.check()
reviews

In [ ]:
#q4.hint()
#q4.solution()

## 5.

Run the cell below to create and display a DataFrame called `animals`:

In [ ]:
animals = pd.DataFrame({'Cows': [12, 20], 'Goats': [22, 19]}, index=['Year 1', 'Year 2'])
animals

In the cell below, write code to save this DataFrame to disk as a csv file with the name `cows_and_goats.csv`.

In [ ]:
# Your code goes here
animals.to_csv('cows_and_goats.csv', sep=',', encoding='utf-8')

# Check your answer
q5.check()

In [ ]:
#q5.hint()
#q5.solution()

# Keep going

Move on to learn about **[indexing, selecting and assigning](https://www.kaggle.com/residentmario/indexing-selecting-assigning)**.

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/pandas/discussion) to chat with other learners.*